# Geo-Stacking Calibration Walkthrough

This notebook validates the sparse matrix construction and dataset creation pipeline for CD-level calibration. It traces a single household through the system to verify correctness.

## Section 1: Setup & Matrix Construction

Build the sparse calibration matrix `X_sparse` where rows are targets and columns are (household × CD) pairs.

In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np

from policyengine_us import Microsimulation
from policyengine_us_data.storage import STORAGE_FOLDER
from policyengine_us_data.datasets.cps.geo_stacking_calibration.metrics_matrix_geo_stacking_sparse import (
    SparseGeoStackingMatrixBuilder,
)
from policyengine_us_data.datasets.cps.geo_stacking_calibration.calibration_utils import (
    create_target_groups,
)
from policyengine_us_data.datasets.cps.geo_stacking_calibration.household_tracer import HouseholdTracer
from policyengine_us_data.datasets.cps.geo_stacking_calibration.create_sparse_cd_stacked import create_sparse_cd_stacked_dataset

rng_ben = np.random.default_rng(seed=42)

/home/baogorek/envs/pe/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TEST_LITE == False


In [2]:
db_path = STORAGE_FOLDER / "policy_data.db"
db_uri = f"sqlite:///{db_path}"
builder = SparseGeoStackingMatrixBuilder(db_uri, time_period=2023)

engine = create_engine(db_uri)

query = """
SELECT DISTINCT sc.value as cd_geoid
FROM strata s
JOIN stratum_constraints sc ON s.stratum_id = sc.stratum_id
WHERE s.stratum_group_id = 1
  AND sc.constraint_variable = 'congressional_district_geoid'
ORDER BY sc.value
"""

with engine.connect() as conn:
    result = conn.execute(text(query)).fetchall()
    all_cd_geoids = [row[0] for row in result]

cds_to_calibrate = all_cd_geoids
dataset_uri = STORAGE_FOLDER / "stratified_extended_cps_2023.h5"
sim = Microsimulation(dataset=str(dataset_uri))

In [3]:
targets_df, X_sparse, household_id_mapping = (
    builder.build_stacked_matrix_sparse(
        "congressional_district", cds_to_calibrate, sim
    )
)

target_groups, group_info = create_target_groups(targets_df)
tracer = HouseholdTracer(targets_df, X_sparse, household_id_mapping, cds_to_calibrate, sim)

print(f"X_sparse shape: {X_sparse.shape}")
print(f"Number of target groups: {len(set(target_groups))}")


=== Creating Target Groups ===

National targets (each is a singleton group):
  Group 0: alimony_expense = 12,554,181,166
  Group 1: alimony_income = 12,554,181,166
  Group 2: charitable_deduction = 63,061,583,407
  Group 3: child_support_expense = 31,868,306,036
  Group 4: child_support_received = 31,868,306,036
  Group 5: eitc = 64,440,000,000
  Group 6: health_insurance_premiums_without_medicare_part_b = 371,796,903,749
  Group 7: income_tax = 2,176,481,000,000
  Group 8: interest_deduction = 23,949,514,839
  Group 9: medicaid = 841,806,132,462
  Group 10: medical_expense_deduction = 11,009,051,176
  Group 11: medicare_part_b_premiums = 108,159,099,272
  Group 12: net_worth = 154,512,998,960,600
  Group 13: other_medical_expenses = 268,466,335,694
  Group 14: over_the_counter_health_expenses = 71,220,353,850
  Group 15: person_count_aca_ptc>0 = 19,529,896
  Group 16: person_count_medicaid>0 = 71,644,763
  Group 17: person_count_ssn_card_type=NONE = 12,200,000
  Group 18: qualified_

## Section 2: Understanding the Row Catalog

The tracer provides a catalog of what each row (target) represents. We'll examine Group 71: SNAP Cost (State) - 51 targets across 51 states.

In [4]:
tracer.print_matrix_structure()


MATRIX STRUCTURE BREAKDOWN

Matrix dimensions: 33217 rows × 5889488 columns
  Rows = 33217 targets
  Columns = 13508 households × 436 CDs
           = 13,508 × 436 = 5,889,488

--------------------------------------------------------------------------------
COLUMN STRUCTURE (Households stacked by CD)
--------------------------------------------------------------------------------

Showing first and last 10 CDs of 436 total:

First 10 CDs:
cd_geoid  start_col  end_col  n_households  example_household_id
    1001          0    13507         13508                    25
     101      13508    27015         13508                    25
     102      27016    40523         13508                    25
     103      40524    54031         13508                    25
     104      54032    67539         13508                    25
     105      67540    81047         13508                    25
     106      81048    94555         13508                    25
     107      94556   108063        

In [5]:
group_71 = tracer.get_group_rows(71)
row_loc = group_71.iloc[28]['row_index']
row_info = tracer.get_row_info(row_loc)
var = row_info['variable']
var_desc = row_info['variable_desc']
target_geo_id = int(row_info['geographic_id'])

print("Row info for first SNAP state target:")
row_info

Row info for first SNAP state target:


{'row_index': 33194,
 'variable': 'snap',
 'variable_desc': 'snap_cost_state',
 'geographic_id': '37',
 'geographic_level': 'unknown',
 'target_value': 4041086120.0,
 'stratum_id': 9799,
 'stratum_group_id': 'state_snap_cost'}

In [6]:
state_snap = tracer.row_catalog[
    (tracer.row_catalog['variable'] == row_info['variable']) &
    (tracer.row_catalog['variable_desc'] == row_info['variable_desc'])
].sort_values('geographic_id')

assert state_snap.shape[0] == 51, f"Expected 51 state SNAP targets, got {state_snap.shape[0]}"
state_snap.head(10)

,row_index,variable,variable_desc,geographic_id,geographic_level,target_value,stratum_id,stratum_group_id
33166,33166,snap,snap_cost_state,1,unknown,2.048985e+09,9766,state_snap_cost
33167,33167,snap,snap_cost_state,10,unknown,2.962075e+08,9773,state_snap_cost
33168,33168,snap,snap_cost_state,11,unknown,3.793723e+08,9774,state_snap_cost
33169,33169,snap,snap_cost_state,12,unknown,6.756577e+09,9775,state_snap_cost
33170,33170,snap,snap_cost_state,13,unknown,3.232508e+09,9776,state_snap_cost
33171,33171,snap,snap_cost_state,15,unknown,8.424059e+08,9777,state_snap_cost
33172,33172,snap,snap_cost_state,16,unknown,2.494227e+08,9778,state_snap_cost
33173,33173,snap,snap_cost_state,17,unknown,5.440580e+09,9779,state_snap_cost
33174,33174,snap,snap_cost_state,18,unknown,1.302143e+09,9780,state_snap_cost
33175,33175,snap,snap_cost_state,19,unknown,5.091406e+08,9781,state_snap_cost


## Section 3: Finding an Interesting Household

We need a household with:
- More than one person
- More than one SPM unit
- Each SPM unit has positive SNAP

This tests that we correctly aggregate SNAP at the household level (sum across SPM units, not persons).

In [7]:
entity_rel = pd.DataFrame(
    {
        "person_id": sim.calculate("person_id", map_to="person").values,
        "household_id": sim.calculate("household_id", map_to="person").values,
        "tax_unit_id": sim.calculate("tax_unit_id", map_to="person").values,
        "spm_unit_id": sim.calculate("spm_unit_id", map_to="person").values,
        "family_id": sim.calculate("family_id", map_to="person").values,
        "marital_unit_id": sim.calculate("marital_unit_id", map_to="person").values,
    }
)
entity_rel.head()

,person_id,household_id,tax_unit_id,spm_unit_id,family_id,marital_unit_id
0,2501,25,2501,25001,251.0,20
1,10301,103,10301,103001,1031.0,80
2,12501,125,12501,125001,1251.0,99
3,12502,125,12501,125001,1251.0,101
4,12503,125,12502,125001,1252.0,100


Note: SNAP values differ by entity level due to broadcasting:
- `sim.calculate_dataframe(['spm_unit_id', 'snap'])` - rows are SPM units
- `sim.calculate_dataframe(['household_id', 'snap'])` - rows are households
- Person-level broadcasts the SPM unit's SNAP to each person

In [8]:
p_df = sim.calculate_dataframe(['person_household_id', 'person_id', 'snap'], map_to="person")

hh_stats = p_df.groupby('person_household_id').agg(
    person_count=('person_id', 'nunique'),
    snap_min=('snap', 'min'),
    snap_unique=('snap', 'nunique')
).reset_index()

candidates = hh_stats[(hh_stats.person_count > 1) & (hh_stats.snap_min > 0) & (hh_stats.snap_unique > 1)]
candidates.head(10)

,person_household_id,person_count,snap_min,snap_unique
4592,66231,2,2293.199951,2
5672,80662,2,937.499756,2
5804,82168,3,789.199951,3
6683,91997,3,3592.000000,2
7143,97972,2,789.199951,2
8340,112528,2,3236.500000,2
9491,128839,3,789.199951,2


In [9]:
hh_id = candidates.iloc[3]['person_household_id']
p_df.loc[p_df.person_household_id == hh_id]

,person_household_id,person_id,snap,__tmp_weights
19739,91997,9199706,3592.0,0.0
19740,91997,9199707,4333.5,0.0
19741,91997,9199708,4333.5,0.0


This household has 3 persons across 2 SPM units:
- Person 1, 2: SNAP = 3592.0
- Persons 3: SNAP = 789.2 

In [10]:
hh_snap_goal = 3592.0 + 4333.5

snap_df = sim.calculate_dataframe(['spm_unit_id', 'snap'])
snap_subset = entity_rel.loc[entity_rel.household_id == hh_id]
snap_df.loc[snap_df.spm_unit_id.isin(list(snap_subset.spm_unit_id))]

,spm_unit_id,snap
6989,91997002,3592.0
6990,91997004,4333.5


In [11]:
hh_df = sim.calculate_dataframe(['household_id', 'state_fips'])
hh_loc = np.where(hh_df.household_id == hh_id)[0][0]
hh_one = hh_df.iloc[hh_loc]
hh_home_state = hh_one.state_fips

print(f"Household {hh_id} is from state FIPS {hh_home_state}")
hh_one

Household 91997.0 is from state FIPS 50


household_id    91997
state_fips         50
Name: 6683, dtype: int32

## Section 4: Validate Matrix Values

Each household appears as a column in X_sparse for every CD (436 times). For state-level SNAP targets, the matrix value should be:
- `hh_snap_goal` if the CD is in the household's home state
- `0` if the CD is in a different state

In [12]:
hh_col_lku = tracer.get_household_column_positions(hh_id)

for cd in hh_col_lku.keys():
    hh_away_state = int(cd) // 100
    col_loc = hh_col_lku[cd]
    col_info = tracer.get_column_info(col_loc)
    
    assert col_info['household_id'] == hh_id
    
    value_lku = tracer.lookup_matrix_cell(row_idx=row_loc, col_idx=col_loc)
    assert value_lku['household']['household_id'] == hh_id
    
    metric = value_lku['matrix_value']
    assert X_sparse[row_loc, col_loc] == metric

    if hh_away_state != target_geo_id:
        assert metric == 0, f"Expected 0 for CD {cd} (state {hh_away_state}), got {metric}"
    else:
        assert metric == hh_snap_goal, f"Expected {hh_snap_goal} for CD {cd}, got {metric}"

print(f"All {len(hh_col_lku)} CD column values validated for household {hh_id}")

All 436 CD column values validated for household 91997.0


## Section 5: Create Sparse Dataset from Weights

Test `create_sparse_cd_stacked_dataset` which reconstructs an h5 file from weight vectors. We verify:
1. Household appears in mapping file for CDs with non-zero weight
2. New household IDs correctly map back to originals
3. SNAP values are preserved

In [13]:
n_nonzero = 500000
total_size = X_sparse.shape[1]

w = np.zeros(total_size)
nonzero_indices = rng_ben.choice(total_size, n_nonzero, replace=False)
w[nonzero_indices] = 2

cd1 = '103'
cd2 = '3703'
output_dir = './temp'
w[hh_col_lku[cd1]] = 1.5
w[hh_col_lku[cd2]] = 1.7

In [14]:
output_path = f"{output_dir}/mapping1.h5"
output_file = create_sparse_cd_stacked_dataset(
    w,
    cds_to_calibrate,
    cd_subset=[cd1, cd2],
    dataset_path=str(dataset_uri),
    output_path=output_path,
)

sim_test = Microsimulation(dataset=output_path)
df_test = sim_test.calculate_dataframe([
    'congressional_district_geoid',
    'household_id', 'household_weight', 'snap'])

print(f"Output dataset shape: {df_test.shape}")
assert np.isclose(df_test.shape[0] / 2 * 436, n_nonzero, rtol=0.10)

Processing subset of 2 CDs: 103, 3703...
Output path: ./temp/mapping1.h5

Original dataset has 13,508 households
Extracted weights for 2 CDs from full weight matrix
Total active household-CD pairs: 2,292
Total weight in W matrix: 4,583
Processing CD 3703 (2/2)...

Combining 2 CD DataFrames...
Total households across all CDs: 2,292
Combined DataFrame shape: (7054, 184)

Weights in combined_df BEFORE reindexing:
  HH weight sum: 0.01M
  Person weight sum: 0.01M
  Ratio: 1.00

Reindexing all entity IDs using 25k ranges per CD...
  Created 2,292 unique households across 2 CDs
  Reindexing persons using 25k ranges...
  Reindexing tax units...
  Reindexing SPM units...
  Reindexing marital units...
  Final persons: 7,054
  Final households: 2,292
  Final tax units: 3,252
  Final SPM units: 2,412
  Final marital units: 5,445

Weights in combined_df AFTER reindexing:
  HH weight sum: 0.01M
  Person weight sum: 0.01M
  Ratio: 1.00

Overflow check:
  Max person ID after reindexing: 10,203,635
  

In [15]:
mapping = pd.read_csv(f"{output_dir}/mappings/mapping1_household_mapping.csv")
match = mapping.loc[mapping.original_household_id == hh_id].shape[0]
assert match == 2, f"Household should appear twice (once per CD), got {match}"

hh_mapping = mapping.loc[mapping.original_household_id == hh_id]
hh_mapping

,new_household_id,original_household_id,congressional_district,state_fips
1151,75572,91997,103,1
1152,5200579,91997,3703,37


In [16]:
df_test

,congressional_district_geoid,household_id,household_weight,snap
0,103,75000,2.0,0.0
1,103,75001,2.0,0.0
2,103,75002,2.0,0.0
3,103,75003,2.0,0.0
4,103,75004,2.0,0.0
...,...,...,...,...
2287,3703,5201163,2.0,0.0
2288,3703,5201164,2.0,0.0
2289,3703,5201165,2.0,0.0
2290,3703,5201166,2.0,0.0


In [17]:
df_test_cd1 = df_test.loc[df_test.congressional_district_geoid == int(cd1)]
df_test_cd2 = df_test.loc[df_test.congressional_district_geoid == int(cd2)]

hh_mapping_cd1 = hh_mapping.loc[hh_mapping.congressional_district == int(cd1)]
new_hh_id_cd1 = hh_mapping_cd1['new_household_id'].values[0]

assert hh_mapping_cd1.shape[0] == 1
assert hh_mapping_cd1.original_household_id.values[0] == hh_id

w_hh_cd1 = w[hh_col_lku[cd1]]
assert_cd1_df = df_test_cd1.loc[df_test_cd1.household_id == new_hh_id_cd1]

assert np.isclose(assert_cd1_df.household_weight.values[0], w_hh_cd1, atol=0.001)
assert np.isclose(assert_cd1_df.snap.values[0], hh_snap_goal, atol=0.001)

print(f"CD {cd1}: weight={w_hh_cd1}, snap={assert_cd1_df.snap.values[0]}")

CD 103: weight=1.5, snap=7925.5


In [18]:
hh_mapping_cd2 = hh_mapping.loc[hh_mapping.congressional_district == int(cd2)]
new_hh_id_cd2 = hh_mapping_cd2['new_household_id'].values[0]

assert hh_mapping_cd2.shape[0] == 1
assert hh_mapping_cd2.original_household_id.values[0] == hh_id

w_hh_cd2 = w[hh_col_lku[cd2]]
assert_cd2_df = df_test_cd2.loc[df_test_cd2.household_id == new_hh_id_cd2]

assert np.isclose(assert_cd2_df.household_weight.values[0], w_hh_cd2, atol=0.001)
assert np.isclose(assert_cd2_df.snap.values[0], hh_snap_goal, atol=0.001)

print(f"CD {cd2}: weight={w_hh_cd2}, snap={assert_cd2_df.snap.values[0]}")

CD 3703: weight=1.7, snap=7925.5


### Test: Zero weight excludes household from mapping

In [19]:
w[hh_col_lku[cd2]] = 0

output_path = f"{output_dir}/{cd2}.h5"
output_file = create_sparse_cd_stacked_dataset(
    w,
    cds_to_calibrate,
    cd_subset=[cd2],
    dataset_path=str(dataset_uri),
    output_path=output_path,
)

sim_test = Microsimulation(dataset=output_path)
df_test = sim_test.calculate_dataframe(['household_id', 'household_weight', 'snap'])

cd2_mapping = pd.read_csv(f"{output_dir}/mappings/{cd2}_household_mapping.csv")
match = cd2_mapping.loc[cd2_mapping.original_household_id == hh_id].shape[0]
assert match == 0, f"Household with zero weight should not appear in mapping, got {match}"

print(f"Confirmed: household {hh_id} excluded from CD {cd2} mapping when weight=0")

Processing subset of 1 CDs: 3703...
Output path: ./temp/3703.h5

Original dataset has 13,508 households
Extracted weights for 1 CDs from full weight matrix
Total active household-CD pairs: 1,167
Total weight in W matrix: 2,334
Processing CD 3703 (1/1)...

Combining 1 CD DataFrames...
Total households across all CDs: 1,167
Combined DataFrame shape: (3633, 184)

Weights in combined_df BEFORE reindexing:
  HH weight sum: 0.01M
  Person weight sum: 0.01M
  Ratio: 1.00

Reindexing all entity IDs using 25k ranges per CD...
  Created 1,167 unique households across 1 CDs
  Reindexing persons using 25k ranges...
  Reindexing tax units...
  Reindexing SPM units...
  Reindexing marital units...
  Final persons: 3,633
  Final households: 1,167
  Final tax units: 1,683
  Final SPM units: 1,227
  Final marital units: 2,818

Weights in combined_df AFTER reindexing:
  HH weight sum: 0.01M
  Person weight sum: 0.01M
  Ratio: 1.00

Overflow check:
  Max person ID after reindexing: 10,203,632
  Max perso

## Section 6: End-to-End Validation (X @ w == sim.calculate)

The ultimate test: verify that matrix multiplication `X_sparse @ w` matches what we get from running the simulation on the reconstructed h5 file.

With `freeze_calculated_vars=True`, state-dependent variables like SNAP are saved to the h5 file to prevent recalculation.

In [20]:
total_size = X_sparse.shape[1]
w = np.zeros(total_size)
n_nonzero = 50000
nonzero_indices = rng_ben.choice(total_size, n_nonzero, replace=False)
w[nonzero_indices] = 7
w[hh_col_lku[cd1]] = 11
w[hh_col_lku[cd2]] = 12

assert np.sum(w > 0) <= n_nonzero + 2

In [21]:
output_path = f"{output_dir}/national.h5"
output_file = create_sparse_cd_stacked_dataset(
    w,
    cds_to_calibrate,
    dataset_path=str(dataset_uri),
    output_path=output_path,
    freeze_calculated_vars=True,
)

Processing all 436 congressional districts
Output path: ./temp/national.h5

Original dataset has 13,508 households
Total active household-CD pairs: 50,002
Total weight in W matrix: 350,023
Processing CD 1201 (10/436)...
Processing CD 1211 (20/436)...
Processing CD 1221 (30/436)...
Processing CD 1303 (40/436)...
Processing CD 1313 (50/436)...
Processing CD 1705 (60/436)...
Processing CD 1715 (70/436)...
Processing CD 1808 (80/436)...
Processing CD 201 (90/436)...
Processing CD 2204 (100/436)...
Processing CD 2406 (110/436)...
Processing CD 2508 (120/436)...
Processing CD 2609 (130/436)...
Processing CD 2706 (140/436)...
Processing CD 2904 (150/436)...
Processing CD 3201 (160/436)...
Processing CD 3405 (170/436)...
Processing CD 3503 (180/436)...
Processing CD 3610 (190/436)...
Processing CD 3620 (200/436)...
Processing CD 3704 (210/436)...
Processing CD 3714 (220/436)...
Processing CD 3909 (230/436)...
Processing CD 4004 (240/436)...
Processing CD 409 (250/436)...
Processing CD 4204 (26

In [33]:
sim_test = Microsimulation(dataset=output_path)
hh_snap_df = pd.DataFrame(sim_test.calculate_dataframe([
    "household_id", "household_weight", "congressional_district_geoid", "state_fips", "snap"])
)

assert np.sum(w > 0) == hh_snap_df.shape[0], f"Expected {np.sum(w > 0)} rows, got {hh_snap_df.shape[0]}"
print(hh_snap_df.shape)
print(np.sum(w > 0))
hh_snap_df.head()

(50002, 5)
50002


,household_id,household_weight,congressional_district_geoid,state_fips,snap
0,0,7.0,1001,10,0.0
1,1,7.0,1001,10,0.0
2,2,7.0,1001,10,0.0
3,3,7.0,1001,10,0.0
4,4,7.0,1001,10,0.0


In [36]:
print(geo_1_df.shape)
print(X_sparse[row_loc, :].shape)
print(geo_1_df.head())
print(geo_1_df.tail())

(1598, 5)
(1, 5889488)
       household_id  household_weight  congressional_district_geoid  \
23789       5150000               7.0                          3701   
23790       5150001               7.0                          3701   
23791       5150002               7.0                          3701   
23792       5150003               7.0                          3701   
23793       5150004               7.0                          3701   

       state_fips    snap  
23789          37  1243.5  
23790          37     0.0  
23791          37     0.0  
23792          37     0.0  
23793          37     0.0  
       household_id  household_weight  congressional_district_geoid  \
25382       5475112               7.0                          3714   
25383       5475113               7.0                          3714   
25384       5475114               7.0                          3714   
25385       5475115               7.0                          3714   
25386       5475116        

In [48]:
geo_1_df['col_position'] = np.nan
geo_1_df['X_sparse_value'] = np.nan
geo_1_df['w_value'] = np.nan

for i in range(geo_1_df.shape[0]):
    df_hh_id_new = geo_1_df.iloc[i]['household_id']
    # get the old household id
    

In [40]:
print(f"Target row info: {row_info}")

y_hat = X_sparse @ w

# Ok, but hang on, you have two districts from two different states, but you 
# didn't use them here. The geo should be NC
print(row_info)
print(target_geo_id)

snap_hat_geo1 = y_hat[row_loc]

geo_1_df = hh_snap_df.loc[hh_snap_df.state_fips == target_geo_id]
y_hat_sim = np.sum(geo_1_df.snap.values * geo_1_df.household_weight.values)
print(geo_1_df.shape)

print(f"Matrix multiplication (X @ w)[{row_loc}] = {snap_hat_geo1:,.2f}")
print(f"Simulation sum(snap * weight) for state 1 = {y_hat_sim:,.2f}")

# Check if household counts match
n_matrix = np.sum(X_sparse[row_loc, :].toarray() > 0)
n_sim = (geo_1_df.snap > 0).sum()
print(f"Matrix nonzero: {n_matrix}, Sim nonzero: {n_sim}")

assert np.isclose(y_hat_sim, snap_hat_geo1, atol=10), f"Mismatch: {y_hat_sim} vs {snap_hat_geo1}"
print("\nEnd-to-end validation PASSED")

Target row info: {'row_index': 33194, 'variable': 'snap', 'variable_desc': 'snap_cost_state', 'geographic_id': '37', 'geographic_level': 'unknown', 'target_value': 4041086120.0, 'stratum_id': 9799, 'stratum_group_id': 'state_snap_cost'}
{'row_index': 33194, 'variable': 'snap', 'variable_desc': 'snap_cost_state', 'geographic_id': '37', 'geographic_level': 'unknown', 'target_value': 4041086120.0, 'stratum_id': 9799, 'stratum_group_id': 'state_snap_cost'}
37
(1598, 5)
Matrix multiplication (X @ w)[33194] = 2,895,502.61
Simulation sum(snap * weight) for state 1 = 2,920,930.08
Matrix nonzero: 14574, Sim nonzero: 129
[np.float64(0.0), np.float64(0.0), np.float64(12.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0)]
Weight from matrix columns: 12.0
Weight from sim: 11191.0


AssertionError: Mismatch: 2920930.082221985 vs 2895502.609931946

436

In [40]:
print(f"Target row info: {row_info}")

y_hat = X_sparse @ w

# Ok, but hang on, you have two districts from two different states, but you 
# didn't use them here. The geo should be NC
print(row_info)
print(target_geo_id)

snap_hat_geo1 = y_hat[row_loc]

geo_1_df = hh_snap_df.loc[hh_snap_df.state_fips == target_geo_id]
y_hat_sim = np.sum(geo_1_df.snap.values * geo_1_df.household_weight.values)
print(geo_1_df.shape)

print(f"Matrix multiplication (X @ w)[{row_loc}] = {snap_hat_geo1:,.2f}")
print(f"Simulation sum(snap * weight) for state 1 = {y_hat_sim:,.2f}")

# Check if household counts match
n_matrix = np.sum(X_sparse[row_loc, :].toarray() > 0)
n_sim = (geo_1_df.snap > 0).sum()
print(f"Matrix nonzero: {n_matrix}, Sim nonzero: {n_sim}")

# Check total weights
w_in_state = [w[hh_col_lku[cd]] for cd in hh_col_lku if int(cd)//100 == target_geo_id]
print(w_in_state)
print(f"Weight from matrix columns: {np.sum(w_in_state)}")
print(f"Weight from sim: {geo_1_df.household_weight.sum()}")

assert np.isclose(y_hat_sim, snap_hat_geo1, atol=10), f"Mismatch: {y_hat_sim} vs {snap_hat_geo1}"
print("\nEnd-to-end validation PASSED")

Target row info: {'row_index': 33194, 'variable': 'snap', 'variable_desc': 'snap_cost_state', 'geographic_id': '37', 'geographic_level': 'unknown', 'target_value': 4041086120.0, 'stratum_id': 9799, 'stratum_group_id': 'state_snap_cost'}
{'row_index': 33194, 'variable': 'snap', 'variable_desc': 'snap_cost_state', 'geographic_id': '37', 'geographic_level': 'unknown', 'target_value': 4041086120.0, 'stratum_id': 9799, 'stratum_group_id': 'state_snap_cost'}
37
(1598, 5)
Matrix multiplication (X @ w)[33194] = 2,895,502.61
Simulation sum(snap * weight) for state 1 = 2,920,930.08
Matrix nonzero: 14574, Sim nonzero: 129
[np.float64(0.0), np.float64(0.0), np.float64(12.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0)]
Weight from matrix columns: 12.0
Weight from sim: 11191.0


AssertionError: Mismatch: 2920930.082221985 vs 2895502.609931946

In [40]:
print(f"Target row info: {row_info}")

y_hat = X_sparse @ w

# Ok, but hang on, you have two districts from two different states, but you 
# didn't use them here. The geo should be NC
print(row_info)
print(target_geo_id)

snap_hat_geo1 = y_hat[row_loc]

geo_1_df = hh_snap_df.loc[hh_snap_df.state_fips == target_geo_id]
y_hat_sim = np.sum(geo_1_df.snap.values * geo_1_df.household_weight.values)
print(geo_1_df.shape)

print(f"Matrix multiplication (X @ w)[{row_loc}] = {snap_hat_geo1:,.2f}")
print(f"Simulation sum(snap * weight) for state 1 = {y_hat_sim:,.2f}")

# Check if household counts match
n_matrix = np.sum(X_sparse[row_loc, :].toarray() > 0)
n_sim = (geo_1_df.snap > 0).sum()
print(f"Matrix nonzero: {n_matrix}, Sim nonzero: {n_sim}")

# Check total weights
w_in_state = [w[hh_col_lku[cd]] for cd in hh_col_lku if int(cd)//100 == target_geo_id]
print(w_in_state)
print(f"Weight from matrix columns: {np.sum(w_in_state)}")
print(f"Weight from sim: {geo_1_df.household_weight.sum()}")

assert np.isclose(y_hat_sim, snap_hat_geo1, atol=10), f"Mismatch: {y_hat_sim} vs {snap_hat_geo1}"
print("\nEnd-to-end validation PASSED")

Target row info: {'row_index': 33194, 'variable': 'snap', 'variable_desc': 'snap_cost_state', 'geographic_id': '37', 'geographic_level': 'unknown', 'target_value': 4041086120.0, 'stratum_id': 9799, 'stratum_group_id': 'state_snap_cost'}
{'row_index': 33194, 'variable': 'snap', 'variable_desc': 'snap_cost_state', 'geographic_id': '37', 'geographic_level': 'unknown', 'target_value': 4041086120.0, 'stratum_id': 9799, 'stratum_group_id': 'state_snap_cost'}
37
(1598, 5)
Matrix multiplication (X @ w)[33194] = 2,895,502.61
Simulation sum(snap * weight) for state 1 = 2,920,930.08
Matrix nonzero: 14574, Sim nonzero: 129
[np.float64(0.0), np.float64(0.0), np.float64(12.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0)]
Weight from matrix columns: 12.0
Weight from sim: 11191.0


AssertionError: Mismatch: 2920930.082221985 vs 2895502.609931946

In [ ]:
output_dir = "./temp"
w = np.load('w_cd_20251126_131911.npy')
print(len(w))
print(len(cds_to_calibrate))

print(w)
print(dataset_uri)
output_path = f"{output_dir}/RI.h5"
output_file = create_sparse_cd_stacked_dataset(
    w,
    cds_to_calibrate,
    ['4401', '4402'],
    dataset_path=str(dataset_uri),
    output_path=output_path,
    freeze_calculated_vars=False,
)

for i in range(51):
    row_loc = group_71.iloc[i]['row_index']
    row_info = tracer.get_row_info(row_loc)
    var = row_info['variable']
    var_desc = row_info['variable_desc']
    target_geo_id = int(row_info['geographic_id'])
    if target_geo_id == 44:
        break

print("Row info for first SNAP state target:")
row_info
print(f"Target row info: {row_info}")

y_hat = X_sparse @ w
snap_hat_geo44 = y_hat[row_loc]

sim_test = Microsimulation(dataset=output_path)
hh_snap_df = pd.DataFrame(sim_test.calculate_dataframe([
    "household_id", "household_weight", "congressional_district_geoid", "state_fips", "snap"])
)

geo_44_df = hh_snap_df.loc[hh_snap_df.state_fips == 44]
y_hat_sim = np.sum(geo_44_df.snap.values * geo_44_df.household_weight.values)

print("\nThe calibration dashboard shows and estimate of 393.86M")
print(f"Matrix multiplication (X @ w)[{row_loc}] = {snap_hat_geo44:,.2f}")
print(f"Simulation sum(snap * weight) for state 44 = {y_hat_sim:,.2f}")

assert np.isclose(y_hat_sim, snap_hat_geo44, atol=10), f"Mismatch: {y_hat_sim} vs {snap_hat_geo44}"
print("\nFull Weight from Model fitting - End-to-end validation PASSED")

## Cleanup

In [ ]:
import shutil
import os

if os.path.exists('./temp'):
    shutil.rmtree('./temp')
    print("Cleaned up ./temp directory")

In [ ]:
output_path = f"{output_dir}/3714.h5"
output_file = create_sparse_cd_stacked_dataset(
    w,
    ['3714'],
    dataset_path=str(dataset_uri),
    output_path=output_path,
)